In [ ]:
import mlflow
import pandas as pd

uri = ... # Set your MLflow tracking URI here
mlflow.set_tracking_uri(uri)


In [ ]:

aggregation_mapping = {
    "average": "Avg",
    "max": "Max",
    "common_features": "ComF",
    "wcom": "WCom",
    "topk": "TopK",
}

def get_method_name(run):
    params = run.data.params
    method = params.get('recommender_strategy', 'Unknown')
    if method == "SAE":
        aggregation = params.get('SAE_fusion_strategy', 'Unknown')
        method += "-" + aggregation_mapping.get(aggregation, aggregation)
    return method.replace('_', '-')

def generate_recommendations_table(runs):
    all_rows = []
    for run in runs:        
            group_type = run.data.params.get("group_type", "none")
            approach = get_method_name(run)

            row_key = (approach)
            metrics = {
                (group_type, "G/mean"): run.data.metrics.get("CommonItemsNDCG20/mean"),
                (group_type, "U/mean"): run.data.metrics.get("NDCG20/mean"),
                (group_type, "U/min"): run.data.metrics.get("NDCG20/min"),
                (group_type, "Pop"): run.data.metrics.get("Popularity/mean"),
            }

            all_rows.append((row_key, metrics))

    # Build DataFrame from records
    records = {}
    for key, metrics in all_rows:
        if key not in records:
            records[key] = {}
        records[key].update(metrics)

    df = pd.DataFrame.from_dict(records, orient="index")
    df.index.names = ["Approach"]

    # Sort and reindex columns by dataset then metric
    df = df.sort_index(axis=1, level=[0, 1]).sort_values(
        by=["Approach"]
    )
    
    dataset_order = ["sim", "random", "outlier"]
    metric_order = ["G/mean", "U/min", "U/mean", "Pop"]

    cols = df.columns
    cols = sorted(
        cols,
        key=lambda x: (
            dataset_order.index(x[0]) if x[0] in dataset_order else len(dataset_order),
            metric_order.index(x[1]) if x[1] in metric_order else len(metric_order)
        )
    )
    df = df[cols]

    return df.reset_index()

def highlight_top3_dark_to_light(s):
    # Colors from dark to light
    colors = ['mediumseagreen', 'lightgreen']
    
    # Get sorted unique values in descending order
    top_values = s.nlargest(2).unique()
    
    # Assign background color depending on rank
    styles = ['' for _ in s]
    for rank, value in enumerate(top_values):
        styles = [
            f'background-color: {colors[rank]}' if v == value and styles[i] == '' else styles[i]
            for i, v in enumerate(s)
        ]
    return styles

def highlight_bottom3_dark_to_light(s):
    # Colors from dark to light
    colors = ['mediumblue', 'lightblue', 'paleturquoise']
    
    # Get sorted unique values in ascending order
    bottom_values = s.nsmallest(3).unique()
    
    # Assign background color depending on rank
    styles = ['' for _ in s]
    for rank, value in enumerate(bottom_values):
        styles = [
            f'background-color: {colors[rank]}' if v == value and styles[i] == '' else styles[i]
            for i, v in enumerate(s)
        ]
    return styles

In [19]:
def format_latex(df, highlight_max_cols=None, highlight_min_cols=None, round_digits=3):
    formatted_df = df.copy()
    highlight_max_cols = highlight_max_cols or []
    highlight_min_cols = highlight_min_cols or []

    for col in df.columns:
        col_values = df[col]

        if col in highlight_max_cols:
            top_two = col_values.nlargest(2).values

            def format_cell(val):
                if val == top_two[0]:
                    return f"\\textbf{{{val:.{round_digits}f}}}"
                elif val == top_two[1]:
                    return f"\\underline{{{val:.{round_digits}f}}}"
                else:
                    return f"{val:.{round_digits}f}"

        elif col in highlight_min_cols:
            bottom_two = col_values.nsmallest(2).values

            def format_cell(val):
                if val == bottom_two[0]:
                    return f"\\textbf{{{val:.{round_digits}f}}}"
                elif val == bottom_two[1]:
                    return f"\\underline{{{val:.{round_digits}f}}}"
                else:
                    return f"{val:.{round_digits}f}"
        else:
            def format_cell(val):
                return f"{val}"

        formatted_df[col] = col_values.apply(format_cell)

    return formatted_df

highlight_max_cols = [('sim', 'G/mean'), ('sim', 'U/min'), ('sim', 'U/mean'), ('random', 'G/mean'), ('random', 'U/min'), ('random', 'U/mean'), ('outlier', 'G/mean'), ('outlier', 'U/min'), ('outlier', 'U/mean')]
highlight_min_cols = [('sim', 'Pop'), ('random', 'Pop'), ('outlier', 'Pop')]

def add_significance(df, significance):
    for i, row in df.iterrows():
        for ttype in ["sim", "random", "outlier"]:
            for metric in ["$NDCG_{com}$", "$NDCG_{min}$", "Popularity"]:
                filterr = (significance['Strategy'] == row['Approach'].values[0]) & (significance['Metric'] == metric) & (significance['Group Type'] == ttype)

                is_significant = significance[filterr]['Significant'].values and significance[filterr]['Significant'].values[0]
                if is_significant:
                    df.at[i, (ttype, metric)] = f"{df.at[i, (ttype, metric)]}^*"

# Comparing with other approaches

## Group Recommendations Results for **Similar** groups sorted by MovieLens G/mean

In [ ]:
experiments = ['523100174176986081', '333391697323445885']
        
runs = mlflow.search_runs(
    experiment_ids=[experiments[0]],
    filter_string="params.user_set = 'full' and params.group_set = 'test'",
    output_format="list"
)
table = generate_recommendations_table(runs)


selected_columns = []
group_types = ["sim", "random", "outlier"]
for grouptype in group_types:
    for metric in ["G/mean", "U/mean", "U/min"]:
        selected_columns.append((grouptype, metric))



approach_order = [
    'SAE-Avg',
    'SAE-ComF',
    'SAE-Max',
    'SAE-TopK',
    'SAE-WCom',
    'ELSA',
    'ELSA_INT',
    'ADD',
    'LMS',
    'MPL',
    'EPFuzzDA',
    'GFAR',
]

table = table.sort_values(by='Approach', key=lambda x: x.map({approach: i for i, approach in enumerate(approach_order)}))


In [21]:
table

Approach       sim                                  random            \
                G/mean     U/min    U/mean       Pop    G/mean     U/min   
7    SAE-Avg  0.635918  0.556924  0.698164  0.496354  0.633387  0.541569   
8   SAE-ComF  0.589624  0.518298  0.666416  0.466037  0.561267  0.492159   
9    SAE-Max  0.624611  0.553968  0.690503  0.502176  0.615537  0.537901   
10  SAE-TopK  0.638526  0.558251  0.699364  0.498347  0.638136  0.545982   
11  SAE-WCom  0.630216  0.551937  0.694280  0.482555  0.625290  0.534339   
1       ELSA  0.616997  0.531441  0.693722  0.496014  0.590809  0.494636   
2   ELSA_INT  0.508731  0.452824  0.633645  0.427452  0.430111  0.370422   
0        ADD  0.636823  0.556950  0.699173  0.498267  0.630784  0.539845   
5        LMS  0.621082  0.530578  0.670587  0.479160  0.605941  0.505510   
6        MPL  0.479610  0.462186  0.614641  0.443885  0.418426  0.416642   
3   EPFuzzDA  0.633619  0.555927  0.690361  0.494423  0.622119  0.538728   
4       GFAR  0.561466  0.516006  0.653628  0.465758  0.513304  0.485503   

                         outlier                                
      U/mean       Pop    G/mean     U/min    U/mean       Pop  
7   0.691110  0.546145  0.545637  0.496259  0.669215  0.479514  
8   0.639928  0.487395  0.420865  0.414233  0.576588  0.387508  
9   0.681068  0.542535  0.538108  0.501982  0.655501  0.478014  
10  0.694633  0.559547  0.563814  0.505636  0.678558  0.505121  
11  0.685889  0.540571  0.541927  0.482518  0.667009  0.485838  
1   0.676351  0.526286  0.412367  0.362904  0.597728  0.384248  
2   0.581750  0.413752  0.137320  0.141332  0.409709  0.187079  
0   0.691403  0.543637  0.544296  0.490845  0.668927  0.472185  
5   0.647094  0.508674  0.448257  0.416438  0.569069  0.398767  
6   0.574198  0.449292  0.276859  0.320875  0.490138  0.318158  
3   0.677744  0.531106  0.502470  0.477995  0.628842  0.434729  
4   0.622718  0.481629  0.380442  0.421888  0.562638  0.389112

In [5]:
format_latex(
    table,
    highlight_max_cols=highlight_max_cols,
    highlight_min_cols=highlight_min_cols,
    round_digits=3
).to_latex(
    "sae_table.tex",
    index=False,
    float_format="%.3f",
    bold_rows=False,
    column_format="l|rrrr|rrrr|rrrr",
    escape=False,
    caption = (
        "Table showing the performance of different recommendation approaches on the MovieLens dataset."
        "'G/mean' shows the percentage change in mean NDCG@20 using ground-truth recommendations seen by all group members. "
        "'U/min' shows the change in the mean of the minimum NDCG@20 across group members. "
        "'U/mean' shows the change in the mean of the average NDCG@20 across group members. "
        "'Pop' shows the change in the mean popularity of recommended items."
    ),
    label="tab:other:movielens"
)